### Long Short Term Memory
Using LSTM, predicting Company A, B stock prices on 5th day

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
class LightningLTSM(L.LightningModule):
    def __init__(self):
        super().__init__()

        # input size --> number of features/variables 
        # hidden size --> number of output values we want (prediction for day 5)
        self.lstm = nn.LSTM(input_size=1, hidden_size=1)

    def forward(self, input):
        # for an LSTM expected input shape is (sequence_length, batch_size, input_size). However, input is originally (batch_size, sequence_length, input_size)
        input_trans = input.transpose(0, 1)
        # adding a dimension to match the 3 parameters criteria --> in this case, input_size = 1
        input_trans = input_trans.unsqueeze(-1)
        # lstm_out --> will contain each short term memory from each LSTM unit that we unrolled (in this case, it has 4 values) 
        lstm_out, temp = self.lstm(input_trans)
        prediction = lstm_out[-1]
        # to remove the dimensions of size 1 for cleaner outputs
        return prediction.squeeze()
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)
    
    def training_step(self, batch, batch_idx):
        input_i, label_i = batch
        output_i = self.forward(input_i)
        label_i = label_i.squeeze()
        loss = F.mse_loss(output_i, label_i)
        self.log('train_loss', loss)

        if label_i.item() == 0:
            self.log('out_0', output_i)
        else:
            self.log('out_1', output_i)
            
        return loss

In [ ]:
(
    [
        [0., 0.5, 0.25, 1.]
    ]
)

In [3]:
model = LightningLTSM()
print("Let's compare the observed and predicted values: ")
print("Company A: Observed = 0, Predicted =", model(torch.tensor([[0., 0.5, 0.25, 1.]])).detach().item())
print("Company B: Observed = 1, Predicted =", model(torch.tensor([[1., 0.5, 0.25, 1.]])).detach().item())

Let's compare the observed and predicted values: 
Company A: Observed = 0, Predicted = -0.07628270238637924
Company B: Observed = 1, Predicted = -0.08181475847959518


In [7]:
model = LightningLTSM()
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

trainer = L.Trainer(max_epochs=350, log_every_n_steps=2)
trainer.fit(model, train_dataloaders=dataloader)    

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



  | Name | Type | Params | Mode 
--------------------------------------
0 | lstm | LSTM | 16     | train
--------------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode
c:\Users\11ukn\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 349: 100%|██████████| 2/2 [00:00<00:00, 156.73it/s, v_num=3]

`Trainer.fit` stopped: `max_epochs=350` reached.


Epoch 349: 100%|██████████| 2/2 [00:00<00:00, 86.96it/s, v_num=3] 


In [8]:
print("Let's compare the observed and predicted values: ")
print("Company A: Observed = 0, Predicted =", abs(round(model(torch.tensor([[0., 0.5, 0.25, 1.]])).detach().item(), 4)))
print("Company B: Observed = 1, Predicted =", round(model(torch.tensor([[1., 0.5, 0.25, 1.]])).detach().item(), 4))


Let's compare the observed and predicted values: 
Company A: Observed = 0, Predicted = 0.0002
Company B: Observed = 1, Predicted = 0.9932
